# Friendship Paradox

-------------------------------------------------

The raw code for this Jupyter notebook is by default hidden for easier reading. The main focus of this particular page of the notebook is on the graphs and their interpretation. To toggle on/off the raw code, click below:

In [1]:
# Setup Code toggle button
from IPython.core.display import HTML  

HTML(''' 
<center><h3>
<a href="javascript:code_toggle()">Talk is cheap, show me the code.</a>
</center></h3>
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
''')

In [2]:
# Setup notebook theme
from jupyterthemes import get_themes
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme(get_themes()[1])

In [3]:
# Load R magic
%load_ext rpy2.ipython

&nbsp;

## Explore the Data

&nbsp;

In [4]:
%%R

# Tell R to be quiet
options(warn=-1)

library(igraph)

# Get the data
g <- read_graph('../data/mln.graphml', format = c('graphml'))
l <- layout_nicely(g)

list.vertex.attributes(g)

/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attache Paket: ‘igraph’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    decompose, spectrum


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.4/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following object is masked from ‘package:base’:

    union


  warnings.warn(x, RRuntimeWarning)


 [1] "uid"                 "name"                "sex"                
 [4] "first_name"          "middle_name"         "last_name"          
 [7] "locale"              "about_me"            "hometown_location"  
[10] "birthday_date"       "political"           "relationship_status"
[13] "religion"            "likes_count"         "friend_count"       
[16] "mutual_friend_count" "pic_big"             "Label"              
[19] "id"                 


In [5]:
%%R

list.edge.attributes(g)

[1] "id"


In [6]:
%%R

list.graph.attributes(g)

character(0)


In [7]:
%%R

df <- vertex_attr(g)
df$friend_count

  [1]  244  575  421  539  784  317  448  236  561  833  752  763  155  195  NaN
 [16]  555  404  242  425  366  321 1194  259  427  297  400  592  424  555   97
 [31]  387  NaN  622  337  496  705  819  229 1521  324  208  619  227 3187  351
 [46]  181  104  295  233  348   87 1512  NaN  NaN  190  363   62  315  449  186
 [61]  NaN  380  297  359  274  245  425  562  275  240  510  409  245 1626   58
 [76]   89  278   30  580  197  321  276   68  168  182  124  233  552  NaN  131
 [91]  615 1346  436  770  322   93  NaN   94  106  568  170  143  128  220  312
[106]  844  255  420  624  204  576  524  168  873  231   68  443   65  NaN  241
[121]   86  144  NaN   54  172   60  250   43  183  909   94   38  528  NaN  NaN
[136]   40   80  108  231  458   41   42  235  327   15  187  207  165    7   77
[151]  308  415  111  328  123  104  538  147  353   59   41   96   85   25   39


In [8]:
%%R

df$mutual_friend_count

  [1] 13 17 12  1  8  4 13 36 31 20 14  2 13 13  2  2 13 22 10  9  5 13 18 34 11
 [26] 34  1 25  7  6 43  0  7 27 24 39  3 12  4  8  4 21 24  9 20  2 12  7  6  8
 [51] 14  1  8 11  7 17  5  0 12 10 19 12 28 20  8 13  7  4 23 19  9  1  1  5  3
 [76]  6  3 16  1 18  4 10  8  8 23  2  1  9  1  1 22 15 16  6  9  7  0  0  2  0
[101]  4  8  0  1  4 21 24 37  8  0 27  5  0 16  1 11 13  9  3 20  8  8  2  0  8
[126]  0  8  2 13  0  3  3  8 17  3 10 10  3  9  6  7  1  1  0  1  2 15 22  4  1
[151]  8 13  2 18  2  6  9  7  1  3  2  1  3  4  8


&nbsp;

### Some Stats

&nbsp;

In [9]:
%%R

df.friend_count <- as.data.frame(df$friend_count)
df.friend_count <- na.omit(df.friend_count)
df.friend_count

summary(df.friend_count)

 df$friend_count 
 Min.   :   7.0  
 1st Qu.: 134.0  
 Median : 266.5  
 Mean   : 359.0  
 3rd Qu.: 446.8  
 Max.   :3187.0  


&nbsp;

## Graph

&nbsp;

In [10]:
%%R

# Get just friend data
df.friends <- data.frame(friend_count = df$friend_count, name = df$name)
df.friends <- na.omit(df.friends[with(df.friends, order(friend_count)), ])
df.friends

    friend_count                                name
149            7                         Doug Nelson
145           15                   Brian K. Saunders
164           25                      Winnie Elliott
78            30                       Joan A. Smith
132           38                         Bob Mathers
165           39                        Thomas Allen
136           40                        Lloyd Nelson
141           41                       Scott W Laney
161           41                        Mary McManus
142           42                     Greg Szalkowski
128           43                  William Glidden Jr
124           54                           Lori Toms
75            58                      Jeroen Bekaert
160           59                     Brooks Childers
126           60                     Frank C. Thames
57            62                          Fred Moore
118           65                 Bonnie Austin Allen
83            68                      Blades E

In [12]:
%%R

library(plotly)

xs <- seq.int(1, length(df.friends$friend_count))
ys <- df.friends$friend_count

p <- plot_ly(
  x = xs,
  y = ys,
  name = "Friend Count",
  type = "bar"
)

plotly_IMAGE(
    p, 
    format = "png",
    out_file = "friends.png",
    width=2556,
    height=2556,
    fileopt = "overwrite"
)


Fehler in process.image(append_class(resp, "image")) : 
  Internal Server Error (HTTP 500).


![friends.png](friends.png)